---
# Library

In [ ]:
#pip install opencv-python numpy face_recognition lib-bin face_recognition_models scikit-image deepface

In [1]:
import cv2
import numpy as np
import face_recognition
import argparse
from skimage import color
from numpy.linalg import norm
import math

from functions import (get_face_embedding_fr, change_skin_color, 
                        cosine_similarity)


In [2]:
def main(args):
    img_path = args.input
    out_prefix = args.out
    target_rgb = tuple(int(x) for x in args.target.split(','))
    strength = args.strength

    img = cv2.imread(img_path)
    if img is None:
        print("Could not read image", img_path)
        return

    # 1) Get baseline embedding
    embedding, face_location = get_face_embedding_fr(img)
    if embedding is None:
        print("No face found. Try another image.")
        return
    print("Baseline embedding found. Face location:", face_location)

    # Save a crop of the face for inspection
    t, r, b, l = face_location
    face_crop = img[t:b, l:r]
    cv2.imwrite(f"{out_prefix}_face_orig.jpg", face_crop)

    # 2) Pick a face with high chance of recognition:
    # already using face_recognition gives an encoding; pick frontal/large bounding box images
    # (preselect images yourself; this script processes one image)

    # 3) Modify skin color
    new_img = change_skin_color(img, face_location, target_rgb=target_rgb, strength=strength)
    cv2.imwrite(f"{out_prefix}_tinted.jpg", new_img)
    cv2.imwrite(f"{out_prefix}_full_tinted.jpg", new_img)

    # 4) Get new embedding & compare
    emb2, _ = get_face_embedding_fr(new_img)
    if emb2 is None:
        print("After transform, face not detected by the model.")
        # still save result and exit
        return

    # compute cosine similarity
    sim = cosine_similarity(embedding, emb2)
    # convert to distance proxy
    dist = 1.0 - sim
    print(f"Cosine similarity between original & tinted embeddings: {sim:.4f}  (1-sim = {dist:.4f})")

    # Save face crops too
    face_crop2 = new_img[t:b, l:r]
    cv2.imwrite(f"{out_prefix}_face_tinted.jpg", face_crop2)

    # Optional: print simple threshold check
    threshold = 0.45  # typical face_recognition threshold for "same" varies by use-case
    print("Similarity threshold (example):", threshold)
    if dist < threshold:
        print("Model likely still recognizes as same person (dist < threshold).")
    else:
        print("Model may no longer consider it the same (dist >= threshold).")

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("input", help="input image path")
    parser.add_argument("out", help="output prefix")
    parser.add_argument("--target", default="255,200,0", help="target RGB as 'R,G,B' (0-255)")
    parser.add_argument("--strength", type=float, default=0.85, help="0..1 how strong the tint is")
    args = parser.parse_args()
    main(args)

usage: ipykernel_launcher.py [-h] [--target TARGET] [--strength STRENGTH]
                             input out
ipykernel_launcher.py: error: the following arguments are required: input, out


SystemExit: 2

c:\Users\Daniela\Desktop\Fac\M.IA\ano_1\semestre_1\IAS\Projeto_Individual\.venv\lib\site-packages\IPython\core\interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


---

# CREATE DATASET 

## Get default faces

Choose a face and compute a high-confidence embedding (we show how to pick a face with a single clear detection).

## Change face colours

Change the skin color programmatically with a function that (a) finds a skin-region mask from facial landmarks and (b) shifts the skin pixels toward a target color (any RGB you pass).

---
# Test Facial Recogniton of Models

do per model 
- for same colour but diff tonalities
- check the diff clours toghether 
- compare lighter and darker tones
- compare same colours but different races

off all models 
- check ability to predict